In [ ]:
import bolift
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from langchain.prompts.prompt import PromptTemplate


data_path = "paper/data/regression_dataset_sial_ratio.csv"

In [ ]:
raw_data = pd.read_csv(data_path)
raw_data.head()

In [ ]:
asktell = bolift.AskTellFewShot(
    x_formatter=lambda x: f"A synthesis procedure:\n{x}",
    y_name="Si/Al ratio",
    y_formatter=lambda y: f"{y:.2f}",
    model="text-curie-001",
    selector_k=4,
)

In [ ]:
train = np.random.choice(raw_data.shape[0], 450, replace=False)
test = np.setdiff1d(np.arange(raw_data.shape[0]), train)
# shuffle test
np.random.shuffle(test)
for i in train:
    asktell.tell(raw_data.iloc[i, 0], float(raw_data.iloc[i, 1]))
y = []
yhat = []
for j in test[:5]:
    y.append(float(raw_data.iloc[j, 1]))
    yhat.append(asktell.predict(raw_data.iloc[j, 0]))

In [ ]:
modes = [yhat[i].mode() for i in range(len(yhat))]
std = [yhat[i].std() for i in range(len(yhat))]
plt.errorbar(y, modes, yerr=std, fmt="o")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

In [ ]:
for yh, y in zip(yhat, y):
    print(yh, y)

In [ ]:
print(raw_data.iloc[test[0], 1])
print(
    asktell.prompt.format(
        x=asktell._x_formatter(raw_data.iloc[test[0], 0]), i="5", y_name="Si/Al Ratio"
    )
)